In [171]:
from typing import Callable, List

import tenacity
from langchain.output_parsers import RegexParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

In [172]:
class UpdatedPrompt(BaseModel):
    prompt: str = Field(..., description="The updated prompt")

class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        most_recent_message = self.message_history[-1]
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join([most_recent_message] + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

In [173]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        self.current_prompt = ""

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        self.current_prompt = message
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()
        output_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        self.current_prompt = output_parser.parse(message).prompt

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, self.current_prompt)

        # 4. increment time
        self._step += 1

        return speaker.name, self.current_prompt

In [205]:
class Bid(BaseModel):
    bid: int = Field(..., description="The agents numerical bid")

class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        output_parser = PydanticOutputParser(pydantic_object=Bid)
        prompt = PromptTemplate(
            template=self.bidding_template,
            input_variables=["message_history", "recent_message"],
        )
        chain = prompt | self.model | output_parser
        for _ in range(3):
            try:
                output = chain.invoke({"message_history": self.message_history, "recent_message": self.message_history[-1]})
                return output.bid
            except Exception as e:
                print(e)
        return 0

In [206]:
character_names = ["Software Engineer", "Software Architect", "Software Tester"]
prompt = "Complete the following function provided its name and docstring:"
word_limit = 50

In [207]:
game_description = f"""The task is prompt optimisation.
The advisor roles are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each advisor."
)


def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a detailed description of the advisor, {character_name}, in {word_limit} words or less, that emphasizes their skills and responsibilities. 
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description


def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your position is {character_name}.
You help optimise prompts.
Your description is as follows: {character_description}
You are optimsing the most recent prompt.
Your goal is to optimise the prompt such that it is an effective input to a chatbot. 
"""

def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will think and speak in the style of an expert {character_name}, in accordance to your description.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
You will make carefully considered improvements to the prompt, but you must not remove text in curly braces.

{PydanticOutputParser(pydantic_object=UpdatedPrompt).get_format_instructions()}

Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]

In [208]:
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")



Software Engineer Description:

Software Engineer, as a key team member, you excel in designing, developing, and implementing software solutions. Your expertise in programming languages and problem-solving skills drive the success of projects. You diligently write, test, and maintain code to meet high-quality standards and fulfill project requirements.

The task is prompt optimisation.
The advisor roles are: Software Engineer, Software Architect, Software Tester.
Your position is Software Engineer.
You help optimise prompts.
Your description is as follows: Software Engineer, as a key team member, you excel in designing, developing, and implementing software solutions. Your expertise in programming languages and problem-solving skills drive the success of projects. You diligently write, test, and maintain code to meet high-quality standards and fulfill project requirements.
You are optimsing the most recent prompt.
Your goal is to optimise the prompt such that it is an effective input

In [209]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is no improvement needed and 10 is extreme improvements needed, rate how much the following prompt needs improving.

```
{{recent_message}}
```

{PydanticOutputParser(pydantic_object=Bid).get_format_instructions()}

Do nothing else.
    """
    return bidding_template


character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [210]:
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

Software Engineer Bidding Template:
The task is prompt optimisation.
The advisor roles are: Software Engineer, Software Architect, Software Tester.
Your position is Software Engineer.
You help optimise prompts.
Your description is as follows: Software Engineer, as a key team member, you excel in designing, developing, and implementing software solutions. Your expertise in programming languages and problem-solving skills drive the success of projects. You diligently write, test, and maintain code to meet high-quality standards and fulfill project requirements.
You are optimsing the most recent prompt.
Your goal is to optimise the prompt such that it is an effective input to a chatbot. 


```
{message_history}
```

On the scale of 1 to 10, where 1 is no improvement needed and 10 is extreme improvements needed, rate how much the following prompt needs improving.

```
{recent_message}
```

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an exam

In [211]:
# @tenacity.retry(
#     stop=tenacity.stop_after_attempt(2),
#     wait=tenacity.wait_none(),  # No waiting time between retries
#     retry=tenacity.retry_if_exception_type(ValueError),
#     before_sleep=lambda retry_state: print(
#         f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
#     ),
#     retry_error_callback=lambda retry_state: 0,
# )  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid = agent.bid()
    return bid

In [212]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

In [213]:
characters = []
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [214]:
max_iters = 10
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Debate Moderator", prompt)
print(f"(Debate Moderator): {prompt}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Debate Moderator): Complete the following function provided its name and docstring:


'Input to PromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'message_history\', \'recent_message\'] Received: [\'message_history\', \'recent_message\']'
'Input to PromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'message_history\', \'recent_message\'] Received: [\'message_history\', \'recent_message\']'
'Input to PromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'message_history\', \'recent_message\'] Received: [\'message_history\', \'recent_message\']'
'Input to PromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'message_history\', \'recent_message\'] Received: [\'message_history\', \'recent_message\']'
'Input to PromptTemplate is missing variables {\'"properties"\', 

OutputParserException: Failed to parse UpdatedPrompt from completion {"properties": {"prompt": {"title": "Prompt", "description": "Provide the necessary implementation for the function based on its name and docstring.", "type": "string"}}, "required": ["prompt"]}. Got: 1 validation error for UpdatedPrompt
prompt
  field required (type=value_error.missing)

In [ ]:
print(simulator.current_prompt)

Refine the following function provided its name and docstring: [insert function details here]. Optimize the function for streamlined testing and validation procedures to uphold superior software quality.
